In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("C:\\Users\\KIIT\\Desktop\\Project\\Movie-Recommender-System-master\\sid\\movie_metadata.csv")

In [3]:
data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [4]:
#checking the null values
data.isnull().sum(axis=0)

color                         19
director_name                104
num_critic_for_reviews        50
duration                      15
director_facebook_likes      104
actor_3_facebook_likes        23
actor_2_name                  13
actor_1_facebook_likes         7
gross                        884
genres                         0
actor_1_name                   7
movie_title                    0
num_voted_users                0
cast_total_facebook_likes      0
actor_3_name                  23
facenumber_in_poster          13
plot_keywords                153
movie_imdb_link                0
num_user_for_reviews          21
language                      12
country                        5
content_rating               303
budget                       492
title_year                   108
actor_2_facebook_likes        13
imdb_score                     0
aspect_ratio                 329
movie_facebook_likes           0
dtype: int64

In [5]:

data['actor_2_name'] = data['actor_2_name'].replace(np.nan, 'unknown')
data['actor_1_name'] = data['actor_1_name'].replace(np.nan, 'unknown')
data['actor_3_name'] = data['actor_3_name'].replace(np.nan, 'unknown')

In [6]:
data['director_name'] = data['director_name'].replace(np.nan, 'unknown')

In [7]:
data['genres'].head()

0    Action|Adventure|Fantasy|Sci-Fi
1           Action|Adventure|Fantasy
2          Action|Adventure|Thriller
3                    Action|Thriller
4                        Documentary
Name: genres, dtype: object

In [8]:
# Now converting the ‘movie_title’ columns values to lowercase for searching simplicity.
data['movie_title'] = data['movie_title'].str.lower()

In [9]:
# all the movie_title values have a special character added to the end
# which needs to be removed
data['movie_title'][0]

'avatar\xa0'

In [10]:
data['movie_title'].head()

0                                              avatar 
1            pirates of the caribbean: at world's end 
2                                             spectre 
3                               the dark knight rises 
4    star wars: episode vii - the force awakens    ...
Name: movie_title, dtype: object

In [11]:
data['movie_title'] = data['movie_title'].str[:-1]

In [12]:
data['comb'] = data['actor_2_name']+' '+data['actor_1_name']+' '+data['actor_3_name']+' '+data['genres']+' '+data['director_name']

In [13]:
data['comb'].head()

0    Joel David Moore CCH Pounder Wes Studi Action|...
1    Orlando Bloom Johnny Depp Jack Davenport Actio...
2    Rory Kinnear Christoph Waltz Stephanie Sigman ...
3    Christian Bale Tom Hardy Joseph Gordon-Levitt ...
4    Rob Walker Doug Walker unknown Documentary Dou...
Name: comb, dtype: object

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
#Creating count matrix now
cv = CountVectorizer()
count_matrix = cv.fit_transform(data['comb'])

#and now similarity score matrix
sim = cosine_similarity(count_matrix)

In [16]:
sim

array([[1.        , 0.24174689, 0.16116459, ..., 0.0805823 , 0.        ,
        0.        ],
       [0.24174689, 1.        , 0.18181818, ..., 0.        , 0.        ,
        0.        ],
       [0.16116459, 0.18181818, 1.        , ..., 0.09090909, 0.        ,
        0.        ],
       ...,
       [0.0805823 , 0.        , 0.09090909, ..., 1.        , 0.0836242 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.0836242 , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [17]:
#movie user like
m = 'Avatar'
m = m.lower()

In [18]:
m in data['movie_title'].unique()

True

In [19]:
i = data.loc[data['movie_title']==m].index[0]
print(i)

0


In [20]:
lst = list(enumerate(sim[i]))

In [21]:
#sorting the list
# in the descending order of similarity score
lst = sorted(lst, key = lambda x:x[1], reverse=True)

In [22]:
# as we need only 10 most similar movies
#keeping only top 10 values of list
# not keeping the first index '0
# because it is the same movie, not similar
lst=lst[1:11]

In [23]:
#let 's create a function
#this funcion takes movie you like as input
#and prints the 10 most relavent movies
def recommend (m) :
    m=m.lower()
    #check if movie is in our database or not
    if m not in data['movie_title'].unique():
        print('This movie is not in our database.')
    else:
        i = data.loc[data['movie_title'] == m].index[0]
        lst = list(enumerate(sim [i]))
        lst = sorted(lst,key = lambda x:x[1], reverse=True)
        lst = lst[1:11]
        l=[]
        for i in range(len(lst)):
            a=lst[i][0]
            l.append(data['movie_title'][a])
        for i in range(len(l)):
            print(l[i])

In [25]:
recommend('deadpool')

green lantern
x-men origins: wolverine
howard the duck
the ice pirates
agent cody banks 2: destination london
fantastic four
fantastic four
ant-man
blade: trinity
nutty professor ii: the klumps


In [26]:
recommend('antman')

This movie is not in our database.


In [27]:
recommend('ant-man')

captain america: the winter soldier
captain america: the first avenger
the specials
jurassic world
dawn of the planet of the apes
tomorrowland
journey 2: the mysterious island
what planet are you from?
teenage mutant ninja turtles
teenage mutant ninja turtles: out of the shadows


In [28]:
recommend('captain america: the winter soldier')

captain america: civil war
captain america: the first avenger
avengers: age of ultron
the avengers
the avengers
ant-man
fantastic 4: rise of the silver surfer
the island
the island
star trek


In [29]:
recommend('the avengers')

the avengers
the avengers
captain america: civil war
iron man 2
captain america: the winter soldier
iron man 3
star trek
iron man
serenity
the island


In [31]:
recommend('the avengers')

the avengers
the avengers
captain america: civil war
iron man 2
captain america: the winter soldier
iron man 3
star trek
iron man
serenity
the island
